Codificación con base en la media

In [1]:
import pandas as pd
import numpy as np
ruta='data/'

data=pd.read_csv(ruta+'temps.txt',sep='\s+',header=None) # Fuente: http://astro.iam.udg.mx/Estacion/NOAAPRMO.TXT
                                                         #         http://astro.iam.udg.mx/Estacion/NOAAMO.TXT
data.columns=['Temperatura máxima','Temperatura mínima','HDD','CDD','Rain']
d=['Sábado','Domingo','Lunes','Martes','Miércoles','Jueves','Viernes']
dias=[d[i%7] for i in range(len(data))]
catego=['Soleado','Mayormente soleado','Parcialmente nublado','Mayormente nublado','Nublado','Lluvioso']
prono=[catego[-1] if data['Rain'][i]>0 else catego[int(round(1.2*(data['HDD'][i]-data['CDD'][i])+2))] for i in range(len(data))]
df=pd.DataFrame(columns=['Días','Pronóstico','Temperatura máxima','Temperatura mínima'],index=range(len(data)))
df.iloc[:,2:4]=data.iloc[:,0:2]
df['Días']=dias
df['Pronóstico']=prono
df.head()

,Días,Pronóstico,Temperatura máxima,Temperatura mínima
0,Sábado,Parcialmente nublado,28.2,11.2
1,Domingo,Parcialmente nublado,29.1,10.7
2,Lunes,Mayormente nublado,26.1,10.4
3,Martes,Mayormente soleado,29.4,11.1
4,Miércoles,Mayormente soleado,29.8,11.1


In [2]:
df.groupby('Pronóstico')['Temperatura máxima'].mean()

Pronóstico
Lluvioso                27.250000
Mayormente nublado      25.600000
Mayormente soleado      28.228571
Nublado                 25.650000
Parcialmente nublado    27.022222
Soleado                 28.900000
Name: Temperatura máxima, dtype: float64

Forma 1, se usa media pero puede ser cualquier medida de tendencia central

In [3]:
df['media']=df['Pronóstico'].map(_)
df.head()

,Días,Pronóstico,Temperatura máxima,Temperatura mínima,media
0,Sábado,Parcialmente nublado,28.2,11.2,27.022222
1,Domingo,Parcialmente nublado,29.1,10.7,27.022222
2,Lunes,Mayormente nublado,26.1,10.4,25.600000
3,Martes,Mayormente soleado,29.4,11.1,28.228571
4,Miércoles,Mayormente soleado,29.8,11.1,28.228571


Usando los coeficientes de una regresión en una codificación de Efectos

In [5]:
# Elimina primera categoría de Pronóstico
E=pd.get_dummies(df,columns=['Pronóstico'], drop_first=True, prefix='', prefix_sep='')
E.head()

,Días,Temperatura máxima,Temperatura mínima,media,Mayormente nublado,Mayormente soleado,Nublado,Parcialmente nublado,Soleado
0,Sábado,28.2,11.2,27.022222,0,0,0,1,0
1,Domingo,29.1,10.7,27.022222,0,0,0,1,0
2,Lunes,26.1,10.4,25.600000,1,0,0,0,0
3,Martes,29.4,11.1,28.228571,0,1,0,0,0
4,Miércoles,29.8,11.1,28.228571,0,1,0,0,0


In [6]:
# regresa directamente el contenido
CR,=set(df['Pronóstico'].unique())-set(E.columns)
CR

'Lluvioso'

In [7]:
idx,=np.where(df['Pronóstico']==CR)
idx

array([17, 44], dtype=int64)

In [8]:
E.iloc[:,4:]=E.iloc[:,4:].astype('int')
E.dtypes


C:\Users\nhernand\AppData\Local\Temp\ipykernel_21132\940766994.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  E.iloc[:,4:]=E.iloc[:,4:].astype('int')


Días                     object
Temperatura máxima       object
Temperatura mínima       object
media                   float64
Mayormente nublado        int32
Mayormente soleado        int32
Nublado                   int32
Parcialmente nublado      int32
Soleado                   int32
dtype: object

In [9]:
E.loc[idx,4:]=-1

C:\Users\nhernand\AppData\Local\Temp\ipykernel_21132\2179449719.py:1: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  E.loc[idx,4:]=-1


In [10]:
E.iloc[17]

Días                    Martes
Temperatura máxima        27.9
Temperatura mínima        11.5
media                    27.25
Mayormente nublado          -1
Mayormente soleado          -1
Nublado                     -1
Parcialmente nublado        -1
Soleado                     -1
Name: 17, dtype: object

In [11]:
df.Pronóstico.unique()

array(['Parcialmente nublado', 'Mayormente nublado', 'Mayormente soleado',
       'Soleado', 'Lluvioso', 'Nublado'], dtype=object)

In [12]:
df_reg = df[df['Pronóstico'] != 'Lluvioso']
df_reg.Pronóstico.unique()

array(['Parcialmente nublado', 'Mayormente nublado', 'Mayormente soleado',
       'Soleado', 'Nublado'], dtype=object)

In [13]:
from sklearn.linear_model import LinearRegression


lin=LinearRegression()
lin.fit(E[['Parcialmente nublado', 'Mayormente nublado', 'Mayormente soleado',
       'Soleado', 'Nublado']], E['Temperatura máxima'])
lin

LinearRegression()

In [14]:
lin.coef_

array([-0.08624339, -1.50846561,  1.12010582,  1.79153439, -1.45846561])

In [15]:
orden=['Parcialmente nublado', 'Mayormente nublado', 'Mayormente soleado',
       'Soleado', 'Nublado']
d={catego:coef for catego,coef in zip(orden, lin.coef_)}
d # Acá falta lluvioso

{'Parcialmente nublado': -0.08624338624338683,
 'Mayormente nublado': -1.5084656084656092,
 'Mayormente soleado': 1.120105820105819,
 'Soleado': 1.7915343915343904,
 'Nublado': -1.4584656084656071}

In [16]:
# Se agrega lluvioso
d[CR]=lin.intercept_
d

{'Parcialmente nublado': -0.08624338624338683,
 'Mayormente nublado': -1.5084656084656092,
 'Mayormente soleado': 1.120105820105819,
 'Soleado': 1.7915343915343904,
 'Nublado': -1.4584656084656071,
 'Lluvioso': 27.10846560846561}

In [17]:
# se agrega mappeo
# se decide usar el nombre 'efectos' la codificación de estas variables tiene el efecto
# de que el promedio de las categorias es la sumatoria entre la intersección y los coeficientes
# de cada categoría 

df['efectos']=df.Pronóstico.map(d)
df.head()

,Días,Pronóstico,Temperatura máxima,Temperatura mínima,media,efectos
0,Sábado,Parcialmente nublado,28.2,11.2,27.022222,-0.086243
1,Domingo,Parcialmente nublado,29.1,10.7,27.022222,-0.086243
2,Lunes,Mayormente nublado,26.1,10.4,25.600000,-1.508466
3,Martes,Mayormente soleado,29.4,11.1,28.228571,1.120106
4,Miércoles,Mayormente soleado,29.8,11.1,28.228571,1.120106


In [ ]:
# reducción de categorías
# englobar categorías en una sola (feature hashing)
